# Generative adversarial network for financial time series

## Setup

In [1]:
import os 
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#!pip install nomkl

In [2]:
!pip install tensorflow

You should consider upgrading via the '/Users/hyunjimoon/GoogleDrive_hmb/tolzul/DataInDM/Code_Commit/4_Auto/GeomPrior/venv/bin/python3 -m pip install --upgrade pip' command.


In [5]:
!pip install tensorflow

You should consider upgrading via the '/Users/hyunjimoon/GoogleDrive_hmb/tolzul/DataInDM/Code_Commit/4_Auto/GeomPrior/venv/bin/python3 -m pip install --upgrade pip' command.


In [9]:
# Requires the latest pip
!pip install --upgrade pip

# Current stable release for CPU and GPU
!pip install tensorflow

# Or try the preview build (unstable)
#pip install tf-nightly



  Using cached pip-22.2.2-py3-none-any.whl (2.0 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4


In [10]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

ModuleNotFoundError: No module named 'tensorflow'

In [11]:

from tensorflow.keras import layers

import tensorflow_addons as tfa

tf.__version__

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
import os
import time
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
from IPython import display
#import seaborn

## Create the models

In [ ]:
# residual link used in attention
class ResidualLink(layers.Layer):
    def __init__(self):
      super(ResidualLink, self).__init__()
    def build(self, input_shape):  
      self.sigma = tf.Variable(0.0, trainable=True)
    def call(self, x, attn):  
      return x + self.sigma * attn

def trianglular_mask(long,short,shrink,dim):
    mask = np.zeros([long, short], dtype=bool)
    for i in range(long):
      mask[i,max(0,int(i/long*short)-shrink//2):int(i/long*short)+1] = True
    if dim ==3:
      return mask[np.newaxis,shrink:]
    elif dim ==4:
      return mask[np.newaxis,np.newaxis,shrink:]
    else:
      raise ValueError

# multi-head attention block, supports sparse attention
def multi_head_attn_block(x, hidden_ratio, g_ratio, nH, sparse, GD, shrink=0, **kwargs):
    def sparse_mask(long, short, kind):
        '''
        mask for sparse attention, 
        kind from LeftFloorMask, RightFloorMask, LeftRepetitiveMask and RightRepetitiveMask, 
        used in Your Local GAN
        '''
        stride = int(np.sqrt(short))
        assert long % short == 0
        multiple = long//short
        if kind in ['LeftFloorMask', 'RightFloorMask']:
            indices = []
            for row in range(short):
                for col in range(row - (row % stride), row + 1):
                    indices.append([row, col])
            indices = np.array(indices)
            mask = np.zeros([short, short], dtype=bool)
            if kind == 'LeftFloorMask':          
                mask[indices[:, 0], indices[:, 1]] = True 
            else:   
                mask[indices[:, 1], indices[:, 0]] = True 

        if kind in ['LeftRepetitiveMask', 'RightRepetitiveMask']:
            if kind == 'RightRepetitiveMask':
                col_indices = np.arange(0,short,stride)
            else:
                col_indices = np.arange(stride - 1,short,stride)
            mask = np.eye(short, dtype=bool)
            for col in col_indices:
                mask[:,col] = True
        return np.vstack([mask]*multiple)

    def get_grid_masks(long, short, nH):
        return np.repeat(np.array([sparse_mask(long,short,'RightFloorMask'),
        sparse_mask(long,short,'LeftFloorMask'),
        sparse_mask(long,short,'RightRepetitiveMask'),
        sparse_mask(long,short,'LeftRepetitiveMask')]), nH//4, axis=0)

    _, location_num, num_channels = x.shape.as_list()
    assert num_channels % (hidden_ratio*nH) == 0       
    downsampled_num = location_num // 2
    hidden_size = num_channels // hidden_ratio             
    head_size = hidden_size // nH         

    # theta path
    if GD == 'G':
      theta = tfa.layers.SpectralNormalization(layers.Conv1D(hidden_size, 1, strides=1, padding='same', **kwargs))(x[:,shrink:])
      theta = tf.reshape(theta, [-1, location_num-shrink, nH, head_size])
      theta = tf.transpose(theta, [0, 2, 1, 3])
    else:
      theta = tfa.layers.SpectralNormalization(layers.Conv1D(hidden_size, 1, strides=1, padding='same', **kwargs))(x)
      theta = tf.reshape(theta, [-1, location_num, nH, head_size])
      theta = tf.transpose(theta, [0, 2, 1, 3])

    # phi path
    phi = tfa.layers.SpectralNormalization(layers.Conv1D(hidden_size, 1, strides=1, padding='same', **kwargs))(x)
    phi = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding="valid")(phi)
    phi = tf.reshape(phi, [-1, downsampled_num, nH, head_size])       
    phi = tf.transpose(phi, [0, 2, 1, 3])

    attn = tf.matmul(theta, phi, transpose_b=True) #/ tf.math.sqrt(tf.cast(theta.shape[-1], tf.float32))
    if sparse:
      masks = tf.constant(get_grid_masks(location_num, downsampled_num, nH)) # acquire masks
      attn = tf.keras.layers.Softmax()(attn, masks)
    else:
      if GD == 'G':
        masks = tf.constant(trianglular_mask(location_num, downsampled_num, shrink, dim=4)) # acquire masks
        attn = tf.keras.layers.Softmax()(attn, masks)
        attn = attn
      else: 
        attn = tf.keras.layers.Softmax()(attn)

    # g path
    g_hidden = num_channels // g_ratio
    g_head_size = g_hidden // nH
    g = tfa.layers.SpectralNormalization(layers.Conv1D(g_hidden, 1, strides=1, padding='same', **kwargs))(x)
    g = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding="valid")(g)
    g = tf.reshape(g, [-1, downsampled_num, nH, g_head_size])
    # swap for heads
    g = tf.transpose(g, [0, 2, 1, 3])

    attn_g = tf.matmul(attn, g)
    # put heads to the end
    attn_g = tf.transpose(attn_g, [0, 2, 3, 1])
    if GD == 'G':
      attn_g = tf.reshape(attn_g, [-1, location_num-shrink, g_hidden])
    else:
      attn_g = tf.reshape(attn_g, [-1, location_num, g_hidden])
    attn_g = tfa.layers.SpectralNormalization(layers.Conv1D(num_channels, 1, strides=1, padding='same', **kwargs))(attn_g)
    if GD == 'G': x = x[:,shrink:]
    return ResidualLink()(x, attn_g)

# ResNet generator block, used in SAGAN, etc.
def G_res_block(x, out_channels, g_kernel, act=layers.ReLU, **kwargs):
    x_0 = x
    x = layers.BatchNormalization()(x)
    x = act()(x) 
    x = tfa.layers.SpectralNormalization(layers.Conv1D(out_channels, g_kernel, strides=1, padding='valid', **kwargs))(x)
    x = layers.BatchNormalization()(x)
    x = act()(x) 
    x = tfa.layers.SpectralNormalization(layers.Conv1D(out_channels, g_kernel, strides=1, padding='valid', **kwargs))(x)    
    x_0 = tfa.layers.SpectralNormalization(layers.Conv1D(out_channels, 1, strides=1, padding='same', **kwargs))(x_0)  
    return x_0[:,-x.shape[1]:] + x

# ResNet generator output layer
def G_out_sn_block(x, out_channels, g_kernel, act=layers.ReLU, **kwargs):
    x = act()(x) 
    x = tfa.layers.SpectralNormalization(layers.Conv1D(x.shape[-1], 1, strides=1, padding='same', **kwargs))(x)
    x = act()(x) 
    x = tfa.layers.SpectralNormalization(layers.Conv1D(out_channels, 1, strides=1, padding='same', **kwargs))(x)
    return x

# ResNet discrinimtor start block
def D_res_start_block(x, out_channels, d_kernel, bn=True, act=layers.LeakyReLU, **kwargs):
    x_0 = x
    x = tfa.layers.SpectralNormalization(layers.Conv1D(out_channels, d_kernel, strides=1, padding='same', **kwargs))(x)
    x = act()(x) 
    x = tfa.layers.SpectralNormalization(layers.Conv1D(out_channels, d_kernel, strides=2, padding='same', **kwargs))(x)
    x_0 = tfa.layers.SpectralNormalization(layers.Conv1D(out_channels, 2, strides=2, padding='same', **kwargs))(x_0)  
    return x + x_0

# ResNet discrinimtor block    
def D_res_block(x, out_channels, d_kernel, bn=True, act=layers.LeakyReLU, **kwargs):
    x_0 = x
    if bn: x = layers.BatchNormalization()(x)
    x = act()(x)
    x = tfa.layers.SpectralNormalization(layers.Conv1D(out_channels, d_kernel, strides=1, padding='same', **kwargs))(x)
    if bn: x = layers.BatchNormalization()(x)
    x = act()(x)
    x = tfa.layers.SpectralNormalization(layers.Conv1D(out_channels, d_kernel, strides=2, padding='same', **kwargs))(x)
    x_0 = tfa.layers.SpectralNormalization(layers.Conv1D(out_channels, 2, strides=2, padding='same', **kwargs))(x_0)
    return x_0 + x

# ResNet discrinimtor last block
def D_res_keep_block(x, d_kernel, bn=True, act=layers.LeakyReLU, **kwargs):
    input_channels = x.shape.as_list()[-1]
    x_0 = x
    if bn: x = layers.BatchNormalization()(x)
    x = act()(x)
    x = tfa.layers.SpectralNormalization(layers.Conv1D(input_channels, d_kernel, strides=1, padding='same', **kwargs))(x)
    if bn: x = layers.BatchNormalization()(x)
    x = act()(x)
    x = tfa.layers.SpectralNormalization(layers.Conv1D(input_channels, d_kernel, strides=1, padding='same', **kwargs))(x)
    return x_0 + x

### The Generator

In [ ]:
def make_generator_model(data_len, data_channel, noise_len, noise_channel, gf_dim, choice, attn, act=layers.ReLU, g_kernel=5):
    if choice == 'sagan':
        attn_shrink = (noise_len - data_len - 6*2*(g_kernel-1))
        inputs = layers.Input(shape= (noise_len, noise_channel))
        x = tfa.layers.SpectralNormalization(layers.Conv1D(gf_dim, 1, strides=1, padding='same'))(inputs)
        skip = tfa.layers.SpectralNormalization(layers.Conv1D(gf_dim, 1, strides=1, padding='same'))(x[:,-data_len:])
        x = G_res_block(x, gf_dim, g_kernel, act=act) 
        skip += tfa.layers.SpectralNormalization(layers.Conv1D(gf_dim, 1, strides=1, padding='same'))(x[:,-data_len:])
        x = G_res_block(x, gf_dim, g_kernel, act=act) 
        skip += tfa.layers.SpectralNormalization(layers.Conv1D(gf_dim, 1, strides=1, padding='same'))(x[:,-data_len:])
        x = G_res_block(x, gf_dim, g_kernel, act=act) 
        skip += tfa.layers.SpectralNormalization(layers.Conv1D(gf_dim, 1, strides=1, padding='same'))(x[:,-data_len:])
        x = multi_head_attn_block(x, hidden_ratio=attn[0], g_ratio=attn[1], nH=attn[2], sparse=attn[3], GD='G', shrink=attn_shrink) 
        skip += tfa.layers.SpectralNormalization(layers.Conv1D(gf_dim, 1, strides=1, padding='same'))(x[:,-data_len:])
        x = G_res_block(x, gf_dim, g_kernel, act=act) 
        skip += tfa.layers.SpectralNormalization(layers.Conv1D(gf_dim, 1, strides=1, padding='same'))(x[:,-data_len:])
        x = G_res_block(x, gf_dim, g_kernel, act=act)   
        skip += tfa.layers.SpectralNormalization(layers.Conv1D(gf_dim, 1, strides=1, padding='same'))(x[:,-data_len:])
        x = G_res_block(x, gf_dim, g_kernel, act=act)    
        skip += tfa.layers.SpectralNormalization(layers.Conv1D(gf_dim, 1, strides=1, padding='same'))(x)
        y = G_out_sn_block(skip, data_channel, g_kernel, act=act) 
        model = tf.keras.Model(inputs, y)

    elif choice == 'transgan':
        def mlp_block(x, ratio=4):
          x0 = x
          num_channels = x.shape[-1]
          x = tfa.layers.SpectralNormalization(layers.Conv1D(num_channels * ratio, 1, strides=1, padding='same'))(x)
          x = act()(x)
          x = tfa.layers.SpectralNormalization(layers.Conv1D(num_channels, 1, strides=1, padding='same'))(x)
          return x0 + x
        def block(x, shrink):
          x = layers.BatchNormalization()(x)
          x = multi_head_attn_block(x, hidden_ratio=attn[0], g_ratio=attn[1], nH=attn[2], sparse=attn[3], GD='G', shrink=shrink)
          x = layers.BatchNormalization()(x)
          x = mlp_block(x)                     
          return x
        def end_block(x, num_channels):               
          x = act()(x)
          x = tfa.layers.SpectralNormalization(layers.Conv1D(x.shape[-1], 1, strides=1, padding='same'))(x)
          x = act()(x)
          x = tfa.layers.SpectralNormalization(layers.Conv1D(num_channels, 1, strides=1, padding='same'))(x)
          return x   
        inputs = layers.Input(shape= (noise_len, noise_channel))
        x = tfa.layers.SpectralNormalization(layers.Conv1D(gf_dim, 1, strides=1, padding='same'))(inputs)               
        skip = tfa.layers.SpectralNormalization(layers.Conv1D(gf_dim, 1, strides=1, padding='same'))(x[:,-data_len:])
        x = block(x,64 + noise_len - data_len-126)  
        skip = skip + tfa.layers.SpectralNormalization(layers.Conv1D(gf_dim, 1, strides=1, padding='same'))(x[:,-data_len:])
        x = block(x,32)
        skip = skip + tfa.layers.SpectralNormalization(layers.Conv1D(gf_dim, 1, strides=1, padding='same'))(x[:,-data_len:])
        x = block(x,16)
        skip = skip + tfa.layers.SpectralNormalization(layers.Conv1D(gf_dim, 1, strides=1, padding='same'))(x[:,-data_len:])       
        x = block(x,8)
        skip = skip + tfa.layers.SpectralNormalization(layers.Conv1D(gf_dim, 1, strides=1, padding='same'))(x[:,-data_len:])       
        x = block(x,6)
        skip = skip + tfa.layers.SpectralNormalization(layers.Conv1D(gf_dim, 1, strides=1, padding='same'))(x)      
        y = end_block(skip, data_channel) 
        model = tf.keras.Model(inputs, y)
  
    return model

### The Discriminator

In [ ]:
def make_discriminator_model(data_len, data_channel, choice, df_dim, attn, bn=True, act=layers.LeakyReLU, cumsum=False, skip_con=False, diff=False):
    if choice == 'sagan':
        d_kernel = 5 
        max_channel = 160       
        inputs = layers.Input(shape= (data_len, data_channel))
        x = inputs
        if cumsum: 
          x_cumsum = tf.math.cumsum(x, axis=1)  
          x = tf.concat([x, x_cumsum],axis=-1)
        x = D_res_start_block(x, min(max_channel,df_dim), d_kernel, bn=bn, act=act)  
        x = D_res_block(x, min(max_channel,df_dim * 2), d_kernel, bn=bn, act=act)  
        x = multi_head_attn_block(x, hidden_ratio=attn[0], g_ratio=attn[1], nH=attn[2], sparse=attn[3], GD='D')  
        x = D_res_block(x, min(max_channel,df_dim * 4), d_kernel, bn=bn, act=act)  
        x = D_res_block(x, min(max_channel,df_dim * 8), d_kernel, bn=bn, act=act)  
        x = D_res_block(x, min(max_channel,df_dim * 16), d_kernel, bn=bn, act=act)  
        x = D_res_keep_block(x, d_kernel, bn=bn, act=act)
        if bn: x = layers.BatchNormalization()(x)
        x = act()(x)
        x = tf.reduce_sum(x, [1])
        y = tfa.layers.SpectralNormalization(layers.Dense(1))(x)
        model = tf.keras.Model(inputs, y)

    elif choice == 'transgan':
        def mlp_block(x, ratio=4):
          x0 = x
          num_channels = x.shape[-1]
          x = tfa.layers.SpectralNormalization(layers.Conv1D(num_channels * ratio, 1, strides=1, padding='same'))(x)
          x = act()(x)
          x = tfa.layers.SpectralNormalization(layers.Conv1D(num_channels, 1, strides=1, padding='same'))(x)
          return x0 + x
        def block(x):
          x = layers.BatchNormalization()(x)
          x = multi_head_attn_block(x, hidden_ratio=attn[0], g_ratio=attn[1], nH=attn[2], sparse=attn[3], GD='D')
          x = layers.BatchNormalization()(x)
          x = mlp_block(x)       
          return x
        inputs = layers.Input(shape= (data_len, data_channel))
        x = inputs
        if cumsum: 
          x_cumsum = tf.math.cumsum(x, axis=1)  
          x = tf.concat([x, x_cumsum],axis=-1)
        elif diff: 
          x_diff = tf.concat([tf.zeros_like(x[:,:1]), x[:,1:]-x[:,:-1]],axis=1)
          x = tf.concat([x, x_diff],axis=-1)
        x = tfa.layers.SpectralNormalization(layers.Conv1D(df_dim, 1, strides=1, padding='same'))(x) 
        if skip_con: skip = tfa.layers.SpectralNormalization(layers.Conv1D(df_dim, 1))(x)      
        x = block(x)  
        if skip_con: skip += tfa.layers.SpectralNormalization(layers.Conv1D(df_dim, 1))(x)      
        x = block(x) 
        if skip_con: skip += tfa.layers.SpectralNormalization(layers.Conv1D(df_dim, 1))(x)      
        x = block(x)
        if skip_con: skip += tfa.layers.SpectralNormalization(layers.Conv1D(df_dim, 1))(x)      
        x = block(x)
        if skip_con: skip += tfa.layers.SpectralNormalization(layers.Conv1D(df_dim, 1))(x)      
        x = block(x)
        if skip_con: skip += tfa.layers.SpectralNormalization(layers.Conv1D(df_dim, 1))(x)      
        if skip_con: 
        	x = tf.keras.layers.Flatten()(skip)
        else:
        	x = tf.keras.layers.Flatten()(x)
        y = tfa.layers.SpectralNormalization(layers.Dense(1))(x)
        model = tf.keras.Model(inputs, y)

    return model

## Define the loss and optimizers

Define loss functions and optimizers for both models.


In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def ls_loss(a,b):
  return tf.reduce_mean(tf.nn.l2_loss(a - b)) 

### Discriminator loss


In [ ]:
def discriminator_loss(real_output, fake_output, choice, generated_images = None, images= None, discriminator = None):
    if choice == 'dcgan': # standard loss
        real_loss = cross_entropy(tf.ones_like(real_output), real_output) 
        fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
    elif choice == 'wgan_gp':
        real_loss = -tf.reduce_mean(real_output)
        fake_loss = tf.reduce_mean(fake_output)

        alpha = tf.random.uniform([images.shape[0],1,1])
        interpolates = alpha*images + ((1-alpha)*generated_images)
        disc_interpolates = discriminator(interpolates, training=True)
        gradients = tf.gradients(disc_interpolates, interpolates)[0]
        slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=[1]))
        gradient_penalty = tf.reduce_mean((slopes-1)**2)
        lam = 10
        total_loss = real_loss + fake_loss + lam*gradient_penalty
    return total_loss

### Generator loss

In [ ]:
def generator_loss(fake_output, choice, real_output = None):
    if choice == 'wgan_gp':
        loss = -tf.reduce_mean(fake_output)
    elif choice == 'dcgan':
        loss = cross_entropy(tf.ones_like(fake_output), fake_output)
    return loss

### Optimizers
The discriminator and the generator optimizers are different since we will train two networks separately.

In [ ]:
def generator_optimizer_fun(choice):
    if choice == 'wgan_gp':
        return tf.keras.optimizers.Adam(1e-4, beta_1=0., beta_2=0.9)            
    elif choice == 'quantgan':
        return tf.keras.optimizers.Adam(1e-4, beta_1=0.0, beta_2=0.9)

def discriminator_optimizer_fun(choice):
    if choice == 'wgan_gp':
        return tf.keras.optimizers.Adam(5*1e-4, beta_1=0., beta_2=0.9)
    elif choice == 'quantgan':
        return tf.keras.optimizers.Adam(3e-4, beta_1=0.0, beta_2=0.9)   

## Define the training loop


In [ ]:
# alpha = 0.6
# 0, 1
score_function1, score_function2

In [ ]:
def train_step_raw(images, generator, discriminator, generator_optimizer, discriminator_optimizer, choice, pre_trained=None):
    noise = tf.random.normal((images.shape[0],) + generator.input_shape[1:])

    with tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      if choice == 'wgan_gp':
        disc_loss = discriminator_loss(real_output, fake_output, choice, generated_images = generated_images, images= images, discriminator=discriminator)
      else:
        disc_loss = discriminator_loss(real_output, fake_output, choice)

    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)   
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    noise = tf.random.normal((images.shape[0],) + generator.input_shape[1:])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output, choice)

      if pre_trained is not None: 
        pre_trained_output = pre_trained(generated_images, training=True) # 0 or 1 from D2
        gen_loss += generator_loss(pre_trained_output, choice)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)     
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    
    return gen_loss, disc_loss

In [ ]:
def train(dataset, batchsize, generator, discriminator, G_opt, D_opt, test, epochs, choice, pre_trained=None):
  train_step = tf.function(train_step_raw)
  dataset = dataset.astype('float32')
  image_batch = tf.Variable(initial_value=np.zeros((batchsize,)+dataset.shape[1:]), trainable=False, dtype='float32')
  datasize = dataset.shape[0]

  for epoch in range(epochs):
    start = time.time()

    G_list = []; D_list = []

    for i in range(datasize//batchsize):
      image_batch.assign(dataset[np.random.choice(datasize, size=batchsize, replace=False)])
      gen_loss, disc_loss = train_step(image_batch, generator, discriminator, G_opt, D_opt, choice, pre_trained)
      G_list.append(gen_loss)
      D_list.append(disc_loss) 

    G_loss.append(np.mean(G_list))
    D_loss.append(np.mean(D_list))

    for fun, seed in test:
      fun(generator, epoch + 1, seed) 

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

### Generate and save images


In [ ]:
def plot_loss(G_loss, D_loss):
  fig = plt.figure(figsize=(4,4)) 
  fig.suptitle('Development of training losses during training')
  if isinstance(D_loss,np.ndarray) and (len(D_loss.shape)>1): 
    [plt.plot(D_loss_i, label='Discriminator loss') for D_loss_i in D_loss]
  else:
    plt.plot(D_loss, label='Discriminator loss')
  plt.plot(G_loss, label='Generator loss')
  plt.legend()
  plt.show()

## Quant GAN

In [ ]:
# official

prelu_kwargs = {'alpha_initializer':tf.keras.initializers.Constant(value=0.25), 'shared_axes':[1,2]}

def TemporalBlock(x, skip, output_dim, hidden_dim_skip, kernel_size, dilation, num_inner_blocks=2, batch_norm=False, **kwargs):
    drop_left = num_inner_blocks * (kernel_size - 1) * dilation
    in_dim = x.shape[-1]

    x_0 = x

    for i in range(num_inner_blocks):
      init = tf.keras.initializers.RandomUniform(minval=-1/np.sqrt(x.shape[2]*kernel_size), maxval=1/np.sqrt(x.shape[2]*kernel_size))
      inits = {'kernel_initializer': init, 'bias_initializer': init}
      x = tfa.layers.SpectralNormalization(layers.Conv1D(output_dim, kernel_size, strides=1, 
                                                       padding='valid', dilation_rate=dilation, **inits, **kwargs))(x)
      x = layers.PReLU(**prelu_kwargs)(x) 
      if batch_norm:
        x = layers.BatchNormalization()(x)
    
    if in_dim != output_dim: 
      init = tf.keras.initializers.RandomUniform(minval=-1/np.sqrt(x_0.shape[2]*1), maxval=1/np.sqrt(x_0.shape[2]*1))
      inits = {'kernel_initializer': init, 'bias_initializer': init}
      x_0 = tfa.layers.SpectralNormalization(layers.Conv1D(output_dim, 1, strides=1, padding='same', **inits, **kwargs))(x_0) 
    
    x = x + x_0[:,drop_left:]
    init = tf.keras.initializers.RandomUniform(minval=-1/np.sqrt(x.shape[2]*1), maxval=1/np.sqrt(x.shape[2]*1))
    inits = {'kernel_initializer': init, 'bias_initializer': init}
    skip = skip + tfa.layers.SpectralNormalization(layers.Conv1D(hidden_dim_skip, 1, strides=1, padding='same', **inits, **kwargs))(x[:, -skip.shape[1]:])
    
    return layers.PReLU(**prelu_kwargs)(x), skip

def TCN(x, output_dim, hidden_dims, hidden_dim_skip=50, 
        kernel_size=2, num_inner_blocks=2, dilation_factor=2, batch_norm=False, **kwargs):
    drop_left =  sum(num_inner_blocks * (kernel_size - 1) * dilation_factor**min(i,6) for i in range(len(hidden_dims)))
    skip = tf.zeros([tf.shape(x)[0], x.shape[1] - drop_left, hidden_dim_skip])
    
    x, skip = TemporalBlock(x, skip, hidden_dims[0], hidden_dim_skip, 1, 1, num_inner_blocks=2, batch_norm=batch_norm, **kwargs)
    for i, h in enumerate(hidden_dims):
        x, skip = TemporalBlock(x, skip, h, hidden_dim_skip, kernel_size, dilation_factor**min(i,6), num_inner_blocks=2, batch_norm=batch_norm, **kwargs)
    
    y = layers.PReLU(**prelu_kwargs)(skip) 
    init = tf.keras.initializers.RandomUniform(minval=-1/np.sqrt(y.shape[2]*1), maxval=1/np.sqrt(y.shape[2]*1))
    inits = {'kernel_initializer': init, 'bias_initializer': init}
    y = tfa.layers.SpectralNormalization(layers.Conv1D(hidden_dim_skip, 1, strides=1, padding='same', **inits, **kwargs))(y) 
    y = layers.PReLU(**prelu_kwargs)(y) 
    y = tfa.layers.SpectralNormalization(layers.Conv1D(output_dim, 1, strides=1, padding='same', **inits, **kwargs))(y) 
    return y

In [ ]:
def G_quant_gan(data_len, data_channel, noise_len, noise_channel, gf_dim = 50): 
    inputs = layers.Input(shape= (noise_len, noise_channel))
    if noise_len-data_len == 126:
      n_block = 6
    elif noise_len-data_len == (126+256):
      n_block = 8
    else:
      raise Exception
    y = TCN(inputs, data_channel, hidden_dims = n_block*[gf_dim], hidden_dim_skip=gf_dim, batch_norm=True)
    model = tf.keras.Model(inputs, y)
    return model

In [ ]:
def D_quant_gan(data_len, data_channel, df_dim = 50, cumsum=False): 
    inputs = layers.Input(shape= (data_len, data_channel))
    x = inputs
    if cumsum: 
      x_cumsum = tf.math.cumsum(x, axis=1)  
      x = tf.concat([x, x_cumsum],axis=-1)
        
    y = TCN(x, 1, hidden_dims = 6*[df_dim], hidden_dim_skip=df_dim, batch_norm=False)
    y = layers.Flatten()(y)
    model = tf.keras.Model(inputs, y)
    return model

## Real data

In [ ]:
import aria
from aria.components.dataframe_regressor import DataFrameRegressor
from aria.components.fourier_seasonality import FourierSeasonality
from aria.components.linear_trend import LinearTrend
from aria.components.linear_trend_changepoints import LinearTrendChangepoints
from aria.datasets.example import load_data
from aria.models.forward_stepwise_selector import ForwardStepwiseSelector
from aria.models.linear_lasso import LinearLassoModel
from aria.models.linear_ridge import LinearRidgeModel
from aria.utils.evaluation import smape, bic

#logging.getLogger().setLevel('DEBUG')


In [ ]:
from pandas_datareader import data as pdr
import yfinance as yf
from datetime import datetime

def download_prices_from_yf(ticker, start_date, end_date, label_list):
    yf.pdr_override()
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
    return pdr.get_data_yahoo(ticker, start_date, end_date)[label_list]

In [ ]:
sp500 = download_prices_from_yf(start_date='2009-05-01', end_date='2021-01-01',
              ticker='^GSPC', label_list=['Adj Close'])
real_close = sp500['Adj Close']
real_return = np.log(real_close).diff().dropna()
real_mean = real_return.mean()
real_std = real_return.std()
standard_return = (real_return-real_mean)/real_std
sequence_data = np.asarray(standard_return) 

scaled_sp500_diff = sp500.rename(columns = {"Date":"ds", "Adj Close": "y"}).iloc[1:, :]
scaled_sp500_diff['y'] = sequence_data

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
scaled_sp500_diff.iloc[-100:, :].y.plot()

In [ ]:
scaled_sp500_diff.y.plot()

In [ ]:
len(scaled_sp500_diff)

In [ ]:
#df['Date']
df[df.index.name] = df.index
df = df.rename(columns = {"Date":"ds"}, inplace = True)

In [ ]:
df = scaled_sp500_diff

n = int(df.shape[0] * 0.8)
train_df, test_df = df.iloc[:n], df.iloc[n:]

components = [
    LinearTrend(),
    LinearTrendChangepoints(),
    FourierSeasonality(7, 3),
    FourierSeasonality(30.4375, 5),
    FourierSeasonality(365.25, 10),
]

print('RIDGE') 
m = LinearRidgeModel(components=components)
m.fit(train_df)
print(pd.DataFrame(index=m.get_features_list(), data=m.get_params()))
print('SMAPE', smape(test_df['y'], m.predict(test_df)['yhat']))

Ridge (normal prior) has a lower SMAPE (1.71 vs 1.77) hence we use Ridge.

In [ ]:
m_coef = pd.DataFrame({"feature":m.get_features_list(), "coef": m.get_params()})
m_coef[m_coef.feature.str.contains('FS')].plot()
season_coef = m_coef[m_coef.feature.str.contains('FS')]
score = np.abs(np.max(season_coef))/ season_coef.apply(lambda x: np.abs(x))

In [ ]:
season_coef

In [ ]:
[s for s in 
 df[df.City.str.contains('ville',case=False)]

In [ ]:
m_coef.get_loc(m_coef

In [ ]:
m_coef.

## Evaluation

In [ ]:
def recursive_simulator(generator, seed, data_len, noise_len, steps):
  generated_list = []
  for i in range(steps):
    generated_image = generator(seed[:,i*data_len:i*data_len+noise_len], training=False)
    generated_list.append(generated_image)
  return np.concatenate(generated_list, axis=1)

In [ ]:
def several_returns(x, days = [5, 22, 64, 128, 256]):
  cumsum_x = np.cumsum(x,axis=1)
  returns = {'1': np.reshape(x,(-1,))}
  for each in days:
    if each < x.shape[1]:
      returns[str(each)] = np.reshape(cumsum_x[:,each:] - cumsum_x[:,:-each],(-1,))
  return returns

def moments(x):
  return {#'mean':np.mean(x, axis=1), 'std':np.std(x, axis=1), 
          'skew':scipy.stats.skew(x, axis=1), 'kurt':scipy.stats.kurtosis(x, axis=1)}

def var_short_fall(x, alpha=0.05):
  var = np.quantile(x, alpha, axis=1) 
  es = np.where(x <= var[:,np.newaxis], x, 0) 
  es = np.mean(es, axis=1)/alpha
  return {'VaR':var, 'ES':es}

def acf_lev(x, max_lag=64):
  x_mean = x - np.mean(x)
  abs_mean = np.abs(x) - np.mean(np.abs(x))
  sq_mean = x**2 - np.mean(x**2)
  x_var = np.mean(x_mean**2)
  abs_var = np.mean(abs_mean**2)
  sq_var = np.mean(sq_mean**2)
  acf = [np.mean(x_mean[:,i:]*x_mean[:,:-i],axis=1) for i in range(1,max_lag+1)]
  acf = np.array(acf).T/x_var
  
  abs_acf = [np.mean(abs_mean[:,i:]*abs_mean[:,:-i],axis=1) for i in range(1,max_lag+1)]
  abs_acf = np.array(abs_acf).T/abs_var
  
  sq_acf = [np.mean(sq_mean[:,i:]*sq_mean[:,:-i],axis=1) for i in range(1,max_lag+1)]
  sq_acf = np.array(sq_acf).T/sq_var
  
  lev = [np.mean(sq_mean[:,i:]*x_mean[:,:-i],axis=1) for i in range(1,max_lag+1)]
  lev = np.array(lev).T/np.sqrt(x_var*sq_var)
  
  return {'acf':acf, 'abs_acf':abs_acf, 'sq_acf':sq_acf, 'lev':lev}

def set_style(ax):
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)

In [ ]:
class Loss():
  def __init__(self, x_real, real_mean, real_std, max_lag, long_seq=False, simple_loss=True, corr_std=True):
    self.days = [5,20,50,100,200]
    self.real_mean = real_mean
    self.real_std = real_std
    self.max_lag = max_lag
    self.x_real = x_real * self.real_std + self.real_mean
    self.real_statistics = self.statistics(self.x_real)
    self.long_seq = long_seq
    self.loss_history = None
    self.simple_loss = simple_loss
    self.corr_std = corr_std  
  def statistics(self, x):
    return {'returns': several_returns(x,days=self.days), 
            'moments': moments(x),
            'var': var_short_fall(x),
            'acf': acf_lev(x, max_lag=self.max_lag)}
  def compute_loss(self, x_fake):
    if self.real_mean ==0:
      x_fake = (x_fake)/np.std(x_fake)
    else:
      x_fake = (x_fake - np.mean(x_fake))/np.std(x_fake)
    self.x_fake = x_fake * self.real_std + self.real_mean
    self.fake_statistics = self.statistics(self.x_fake)
    loss_summary = {}
    loss_summary['returns_dist'] = {each: scipy.stats.wasserstein_distance(self.fake_statistics['returns'][each],
                                                                self.real_statistics['returns'][each])
                               for each in self.fake_statistics['returns']}
    if not (self.simple_loss or self.long_seq): 
      loss_summary['moments_dist'] = {each: scipy.stats.wasserstein_distance(self.fake_statistics['moments'][each],
                                                                self.real_statistics['moments'][each])
                               for each in self.fake_statistics['moments']} 
    loss_summary['moments_diff'] = {each: np.abs(np.mean(self.fake_statistics['moments'][each])-
                                                 np.mean(self.real_statistics['moments'][each]))
                               for each in self.fake_statistics['moments']}
    if not (self.simple_loss or self.long_seq): 
      loss_summary['var_dist'] = {each: scipy.stats.wasserstein_distance(self.fake_statistics['var'][each],
                                                                self.real_statistics['var'][each])
                               for each in self.fake_statistics['var']}
    if not self.simple_loss: 
      loss_summary['var_diff'] = {each: np.abs(np.mean(self.fake_statistics['var'][each])-
                                                 np.mean(self.real_statistics['var'][each]))
                               for each in self.fake_statistics['var']} 
    loss_summary['acf_l2'] = {each: np.linalg.norm(np.mean(self.fake_statistics['acf'][each],axis=0)-
                                                 np.mean(self.real_statistics['acf'][each],axis=0))
                               for each in self.fake_statistics['acf']}                                                        
    self.loss_summary = loss_summary
    if self.loss_history is None:
      self.loss_history = {k0:{k1:[v1] for k1,v1 in v0.items()} for k0, v0 in self.loss_summary.items()}
    else:
      [v1.append(self.loss_summary[k0][k1]) for k0, v0 in self.loss_history.items() for k1,v1 in v0.items()]

  def plot_loss(self, figsize=None):
    num_of_plots = sum(len(v0) for k0, v0 in self.loss_summary.items())
    if figsize is None: figsize = (16, 4*int(np.ceil(num_of_plots/4)))
    fig, axes = plt.subplots(int(np.ceil(num_of_plots/4)), 4, figsize=figsize)
    axes = np.reshape(axes, (-1,))
    i = 0
    for k0, v0 in self.loss_history.items(): 
      for k1,v1 in v0.items():
        axes[i].plot(v1) 
        axes[i].title.set_text(k0+'_'+k1)
        i +=1
  
  def plot_moments(self, figsize=None):
    if figsize is None: figsize = (8, 4)
    fig, axes = plt.subplots(1, 2, figsize=figsize)
    titles = [#'Mean', 'Standard deviation', 
              'Skewness', 'Kurtosis']

    for i, each in enumerate(self.fake_statistics['moments']):
      axes[i].boxplot([self.real_statistics['moments'][each], self.fake_statistics['moments'][each]])
      axes[i].set_xticklabels(['Historical', 'Synthetic'])
      axes[i].grid()
      set_style(axes[i])   
      axes[i].title.set_text(titles[i])
    # fig.tight_layout()

  def plot_var(self, figsize=None):
    if figsize is None: figsize = (8, 4)
    fig, axes = plt.subplots(1, 2, figsize=figsize)
    titles = ['Value at risk', 'Expected shortfall']

    for i, each in enumerate(self.fake_statistics['var']):
      axes[i].boxplot([self.real_statistics['var'][each], self.fake_statistics['var'][each]])
      axes[i].set_xticklabels(['Historical', 'Synthetic'])
      axes[i].grid()
      set_style(axes[i])   
      axes[i].title.set_text(titles[i])

  def plot_corr(self, figsize=None, corr_std = None):
    if corr_std is not None: self.corr_std = corr_std 
    if figsize is None: figsize = (16, 4)
    fig, axes = plt.subplots(1, 4, figsize=figsize)
    titles = ['Serial ACF', 'ACF of absolute log-returns', 'ACF of square log-returns', 'Leverage effect']
    
    for i, each in enumerate(self.fake_statistics['acf']):
      num = np.arange(1,1+self.max_lag)
      acf_real_mean = np.mean(self.real_statistics['acf'][each],axis=0)
      axes[i].plot(num, acf_real_mean, label='Historical')
      acf_fake_mean = np.mean(self.fake_statistics['acf'][each],axis=0)
      axes[i].plot(num, acf_fake_mean, label='Generated', alpha=0.8)
      axes[i].grid()
      set_style(axes[i])   
      axes[i].set_xlabel('Lags')
      # axes[i].set_ylabel('ACF')
      axes[i].legend()
      axes[i].title.set_text(titles[i])
      if self.corr_std: 
        acf_fake_std = np.std(self.fake_statistics['acf'][each],axis=0)
        ub = acf_fake_mean + acf_fake_std
        lb = acf_fake_mean - acf_fake_std
        axes[i].fill_between(num, ub, lb,
              color='C1', alpha=.3)
        acf_real_std = np.std(self.real_statistics['acf'][each],axis=0)
        ub = acf_real_mean + acf_real_std
        lb = acf_real_mean - acf_real_std
        axes[i].fill_between(num, ub, lb,
              color='C0', alpha=.1)
  
  def plot_curve(self, figsize=None, number = 1):
    if figsize is None: figsize = (4, 4)
    fig, axes = plt.subplots(1, 1, figsize=figsize)

    for i in range(number):
        axes.plot(np.cumsum(self.x_real[i]),'C0',label='Historic' if i==0 else None)
        axes.plot(np.cumsum(self.x_fake[i]),'C1',label='Generated' if i==0 else None)
    axes.grid()
    axes.legend()
    set_style(axes)
    axes.title.set_text('Sample curves')

  def plot_hists(self, figsize=None, textbox=True):
    if128 = 1 if str(self.days[3]) in self.fake_statistics['returns'] else 0
    if figsize is None: figsize = (6*(2+if128), 4)
    fig, axes = plt.subplots(1, 2+if128, figsize=figsize)
    
    self.one_hist(self.real_statistics['returns']['1'], self.fake_statistics['returns']['1'], axes[0], textbox)
    self.one_hist(self.real_statistics['returns'][str(self.days[2])], self.fake_statistics['returns'][str(self.days[2])], axes[1], textbox)
    if if128:
      self.one_hist(self.real_statistics['returns'][str(self.days[3])], self.fake_statistics['returns'][str(self.days[3])], axes[2], textbox)
  
  def plot_all(self):
    self.plot_hists()
    self.plot_corr()
    self.plot_moments()
    self.plot_var()
  
  def one_hist(self, x_real, x_fake, ax, textbox, clip = None):
    if clip is not None:
      x_real_clipped = np.clip(x_real,*clip)
      x_fake_clipped = np.clip(x_fake,*clip)
    else:
      x_real_clipped = x_real
      x_fake_clipped = x_fake

    ax.hist([x_real_clipped, x_fake_clipped],
            bins=50, alpha=1, density=True, label=['Historical', 'Synthetic'])
    ax.grid()
    set_style(ax)
    ax.legend()
    ax.set_ylabel('PDF')
    ax.set_xlabel('Log-Return')

    if textbox:
      self.text_box(x_real, ax, 0.05, 'Historical')
      self.text_box(x_fake, ax, 0.75, 'Generated')

  def text_box(self, x, ax, pos, title):
    textstr = '\n'.join(['{}','mean={:.2f}', 'std={:.2f}', 'skew={:.2f}', 'kurt={:.2f}']).format(
        title, np.mean(x), np.std(x), scipy.stats.skew(x), scipy.stats.kurtosis(x))        
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(pos, 0.70, textstr,
        transform=ax.transAxes,
        fontsize=14,
        verticalalignment='top',
        bbox=props)

## Training

There is no guarantee on the number of training epochs. Usually check the results every 100 epochs.

### TAGAN

In [ ]:
choice = 'sagan' 
opt_choice = 'wgan_gp'
loss_choice = 'wgan_gp'

In [ ]:
gf_dim = 64
g_attn = (1,1,1,False)
df_dim = 8
d_attn = (1,1,1,False)
g_act = layers.ReLU
d_act = layers.LeakyReLU
d_bn = True
cumsum = False
g_kernel=5

In [ ]:
BATCH_SIZE = 512
data_len = 128
data_channel = 1
noise_len = 254
noise_channel = 3

data = np.array([standard_return[t:t + len(standard_return)-data_len] for t in range(0, data_len)]).T[:,:,np.newaxis]
data.shape

In [ ]:
def compare_test(model, epoch, test_input):
  generated_image = generator(test_input, training=False)[:,:,0]
  loss_obj_training.compute_loss(generated_image)
  if (epoch%10 == 0) or (epoch == 1):
    display.clear_output()
    loss_obj_training.plot_loss()
    loss_obj_training.plot_corr()
    plt.show()

num_examples_to_generate_2 = 512
seed2 = tf.random.normal([num_examples_to_generate_2, noise_len, noise_channel])

test = [[compare_test,seed2]]

sample_data_2 = data[np.random.choice(data.shape[0], num_examples_to_generate_2, replace=False),:,0]
loss_obj_training = Loss(sample_data_2, real_mean=real_mean, real_std=real_std, max_lag=64)

In [ ]:
generator = make_generator_model(data_len, data_channel, noise_len, noise_channel, gf_dim=gf_dim, choice=choice, attn=g_attn, act=g_act, g_kernel=g_kernel)

In [ ]:
discriminator = make_discriminator_model(data_len, data_channel, choice, df_dim=df_dim, attn=d_attn, bn=d_bn, act=d_act, cumsum=cumsum)

In [ ]:
generator_optimizer = generator_optimizer_fun(opt_choice)
discriminator_optimizer = discriminator_optimizer_fun(opt_choice)

In [ ]:
G_loss = []; D_loss = [];

#### 200e training

In [ ]:
train(data, BATCH_SIZE, generator, discriminator, generator_optimizer, discriminator_optimizer, test, 200, loss_choice)

#### 200e test

In [ ]:
num_examples_to_generate_4 = 512
steps = 20
total_data_length = data_len * steps
total_noise_length = total_data_length - data_len + noise_len
seed4 = tf.random.normal([num_examples_to_generate_4, total_noise_length, noise_channel])
sample_data_4 = np.array([sequence_data])

In [ ]:
%%time
generated_image4 = recursive_simulator(generator, seed4, data_len, noise_len, steps)[:,:,0]
generated_image4.shape

In [ ]:
loss_obj_long = Loss(sample_data_4, real_mean=real_mean, real_std=real_std, max_lag=250, simple_loss=False, long_seq=True)
loss_obj_long.compute_loss(generated_image4)
# loss_obj_long.plot_all()

In [ ]:
loss_obj_long.loss_summary

#### 300e training

In [ ]:
train(data, BATCH_SIZE, generator, discriminator, generator_optimizer, discriminator_optimizer, test, 100, loss_choice)

#### 300e test

In [ ]:
num_examples_to_generate_4 = 512
steps = 20
total_data_length = data_len * steps
total_noise_length = total_data_length - data_len + noise_len
seed4 = tf.random.normal([num_examples_to_generate_4, total_noise_length, noise_channel])
sample_data_4 = np.array([sequence_data])

In [ ]:
%%time
generated_image4 = recursive_simulator(generator, seed4, data_len, noise_len, steps)[:,:,0]
generated_image4.shape

In [ ]:
loss_obj_long = Loss(sample_data_4, real_mean=real_mean, real_std=real_std, max_lag=250, simple_loss=False, long_seq=True)
loss_obj_long.compute_loss(generated_image4)
loss_obj_long.plot_all()

In [ ]:
loss_obj_long.loss_summary

#### 400e training

In [ ]:
train(data, BATCH_SIZE, generator, discriminator, generator_optimizer, discriminator_optimizer, test, 100, loss_choice)

#### 400e test

In [ ]:
num_examples_to_generate_4 = 512
steps = 20
total_data_length = data_len * steps
total_noise_length = total_data_length - data_len + noise_len
seed4 = tf.random.normal([num_examples_to_generate_4, total_noise_length, noise_channel])
sample_data_4 = np.array([sequence_data])

In [ ]:
%%time
generated_image4 = recursive_simulator(generator, seed4, data_len, noise_len, steps)[:,:,0]
generated_image4.shape

In [ ]:
loss_obj_long = Loss(sample_data_4, real_mean=real_mean, real_std=real_std, max_lag=250, simple_loss=False, long_seq=True)
loss_obj_long.compute_loss(generated_image4)
loss_obj_long.plot_all()

In [ ]:
loss_obj_long.loss_summary

#### 500e training

In [ ]:
train(data, BATCH_SIZE, generator, discriminator, generator_optimizer, discriminator_optimizer, test, 100, loss_choice)

#### 500e test

In [ ]:
num_examples_to_generate_4 = 512
steps = 20
total_data_length = data_len * steps
total_noise_length = total_data_length - data_len + noise_len
seed4 = tf.random.normal([num_examples_to_generate_4, total_noise_length, noise_channel])
sample_data_4 = np.array([sequence_data])

In [ ]:
%%time
generated_image4 = recursive_simulator(generator, seed4, data_len, noise_len, steps)[:,:,0]
generated_image4.shape

In [ ]:
loss_obj_long = Loss(sample_data_4, real_mean=real_mean, real_std=real_std, max_lag=250, simple_loss=False, long_seq=True)
loss_obj_long.compute_loss(generated_image4)
loss_obj_long.plot_all()

In [ ]:
loss_obj_long.loss_summary

### QuantGAN

In [ ]:
choice = 'quantgan' 
opt_choice = 'quantgan'
loss_choice = 'dcgan'
cumsum = False

In [ ]:
gf_dim = 80
df_dim = 80

In [ ]:
BATCH_SIZE = 512
data_len = 127
data_channel = 1
noise_len = 253
noise_channel = 3

data = np.array([standard_return[t:t + len(standard_return)-data_len] for t in range(0, data_len)]).T[:,:,np.newaxis]
data.shape

In [ ]:
def compare_test(model, epoch, test_input):
  generated_image = generator(test_input, training=False)[:,:,0]
  loss_obj_training.compute_loss(generated_image)
  if (epoch%10 == 0) or (epoch == 1):
    display.clear_output()
    loss_obj_training.plot_loss()
    loss_obj_training.plot_corr()
    plt.show()

num_examples_to_generate_2 = 512
seed2 = tf.random.normal([num_examples_to_generate_2, noise_len, noise_channel])

test = [[compare_test,seed2]]

sample_data_2 = data[np.random.choice(data.shape[0], num_examples_to_generate_2, replace=False),:,0]
loss_obj_training = Loss(sample_data_2, real_mean=real_mean, real_std=real_std, max_lag=64)

In [ ]:
generator = G_quant_gan(noise_len=noise_len, data_len=data_len, data_channel = data_channel, noise_channel = noise_channel, gf_dim=gf_dim)

In [ ]:
discriminator = D_quant_gan(data_len = data_len, data_channel = data_channel, df_dim=df_dim, cumsum = cumsum)

In [ ]:
generator_optimizer = generator_optimizer_fun(opt_choice)
discriminator_optimizer = discriminator_optimizer_fun(opt_choice)

In [ ]:
G_loss = []; D_loss = [];

#### 200e training

In [ ]:
train(data, BATCH_SIZE, generator, discriminator, generator_optimizer, discriminator_optimizer, test, 200, loss_choice)

#### 200e test

In [ ]:
num_examples_to_generate_4 = 512
steps = 20
total_data_length = data_len * steps
total_noise_length = total_data_length - data_len + noise_len
seed4 = tf.random.normal([num_examples_to_generate_4, total_noise_length, noise_channel])
sample_data_4 = np.array([sequence_data])

In [ ]:
%%time
generated_image4 = recursive_simulator(generator, seed4, data_len, noise_len, steps)[:,:,0]
generated_image4.shape

In [ ]:
loss_obj_long = Loss(sample_data_4, real_mean=real_mean, real_std=real_std, max_lag=250, simple_loss=False, long_seq=True)
loss_obj_long.compute_loss(generated_image4)
# loss_obj_long.plot_all()

In [ ]:
loss_obj_long.loss_summary

#### 300e training

In [ ]:
train(data, BATCH_SIZE, generator, discriminator, generator_optimizer, discriminator_optimizer, test, 100, loss_choice)

#### 300e test

In [ ]:
num_examples_to_generate_4 = 512
steps = 20
total_data_length = data_len * steps
total_noise_length = total_data_length - data_len + noise_len
seed4 = tf.random.normal([num_examples_to_generate_4, total_noise_length, noise_channel])
sample_data_4 = np.array([sequence_data])

In [ ]:
%%time
generated_image4 = recursive_simulator(generator, seed4, data_len, noise_len, steps)[:,:,0]
generated_image4.shape

In [ ]:
loss_obj_long = Loss(sample_data_4, real_mean=real_mean, real_std=real_std, max_lag=250, simple_loss=False, long_seq=True)
loss_obj_long.compute_loss(generated_image4)
loss_obj_long.plot_all()

In [ ]:
loss_obj_long.loss_summary

#### 400e training

In [ ]:
train(data, BATCH_SIZE, generator, discriminator, generator_optimizer, discriminator_optimizer, test, 100, loss_choice)

#### 400e test

In [ ]:
num_examples_to_generate_4 = 512
steps = 20
total_data_length = data_len * steps
total_noise_length = total_data_length - data_len + noise_len
seed4 = tf.random.normal([num_examples_to_generate_4, total_noise_length, noise_channel])
sample_data_4 = np.array([sequence_data])

In [ ]:
%%time
generated_image4 = recursive_simulator(generator, seed4, data_len, noise_len, steps)[:,:,0]
generated_image4.shape

In [ ]:
loss_obj_long = Loss(sample_data_4, real_mean=real_mean, real_std=real_std, max_lag=250, simple_loss=False, long_seq=True)
loss_obj_long.compute_loss(generated_image4)
loss_obj_long.plot_all()

In [ ]:
loss_obj_long.loss_summary

#### 500e training

In [ ]:
train(data, BATCH_SIZE, generator, discriminator, generator_optimizer, discriminator_optimizer, test, 100, loss_choice)

#### 500e test

In [ ]:
num_examples_to_generate_4 = 512
steps = 20
total_data_length = data_len * steps
total_noise_length = total_data_length - data_len + noise_len
seed4 = tf.random.normal([num_examples_to_generate_4, total_noise_length, noise_channel])
sample_data_4 = np.array([sequence_data])

In [ ]:
%%time
generated_image4 = recursive_simulator(generator, seed4, data_len, noise_len, steps)[:,:,0]
generated_image4.shape

In [ ]:
loss_obj_long = Loss(sample_data_4, real_mean=real_mean, real_std=real_std, max_lag=250, simple_loss=False, long_seq=True)
loss_obj_long.compute_loss(generated_image4)
loss_obj_long.plot_all()

In [ ]:
loss_obj_long.loss_summary

### TTGAN

In [ ]:
choice = 'transgan' 
opt_choice = 'wgan_gp'
loss_choice = 'wgan_gp'

In [ ]:
gf_dim = 64
g_attn = (2,2,4,False)
df_dim = 64
d_attn = (2,2,4,True)
g_act = lambda: tf.keras.activations.gelu
d_act = lambda: tf.keras.activations.gelu
d_bn = True
cumsum = True
skip_con = False

In [ ]:
BATCH_SIZE = 128
data_len = 128
data_channel = 1
noise_len = 254
noise_channel = 3

data = np.array([standard_return[t:t + len(standard_return)-data_len] for t in range(0, data_len)]).T[:,:,np.newaxis]
data.shape

In [ ]:
def compare_test(model, epoch, test_input):
  generated_image = generator(test_input, training=False)[:,:,0]
  loss_obj_training.compute_loss(generated_image)
  if (epoch%10 == 0) or (epoch == 1):
    display.clear_output()
    loss_obj_training.plot_loss()
    loss_obj_training.plot_corr()
    plt.show()

num_examples_to_generate_2 = 512
seed2 = tf.random.normal([num_examples_to_generate_2, noise_len, noise_channel])

test = [[compare_test,seed2]]

sample_data_2 = data[np.random.choice(data.shape[0], num_examples_to_generate_2, replace=False),:,0]
loss_obj_training = Loss(sample_data_2, real_mean=real_mean, real_std=real_std, max_lag=64)

In [ ]:
generator = make_generator_model(data_len, data_channel, noise_len, noise_channel, gf_dim=gf_dim, choice=choice, attn=g_attn, act=g_act)

In [ ]:
discriminator = make_discriminator_model(data_len, data_channel, choice, df_dim=df_dim, attn=d_attn, bn=d_bn, act=d_act, cumsum=cumsum, skip_con=skip_con)

In [ ]:
generator_optimizer = generator_optimizer_fun(opt_choice)
discriminator_optimizer = discriminator_optimizer_fun(opt_choice)

In [ ]:
G_loss = []; D_loss = [];

#### 200e training

In [ ]:
train(data, BATCH_SIZE, generator, discriminator, generator_optimizer, discriminator_optimizer, test, 200, loss_choice)

#### test

In [ ]:
num_examples_to_generate_4 = 512
steps = 20
total_data_length = data_len * steps
total_noise_length = total_data_length - data_len + noise_len
seed4 = tf.random.normal([num_examples_to_generate_4, total_noise_length, noise_channel])
sample_data_4 = np.array([sequence_data])

In [ ]:
%%time
generated_image4 = recursive_simulator(generator, seed4, data_len, noise_len, steps)[:,:,0]
generated_image4.shape

In [ ]:
loss_obj_long = Loss(sample_data_4, real_mean=real_mean, real_std=real_std, max_lag=250, simple_loss=False, long_seq=True)
loss_obj_long.compute_loss(generated_image4)
loss_obj_long.plot_all()

In [ ]:
loss_obj_long.loss_summary

#### 500e training

In [ ]:
train(data, BATCH_SIZE, generator, discriminator, generator_optimizer, discriminator_optimizer, test, 300, loss_choice)

#### test

In [ ]:
num_examples_to_generate_4 = 512
steps = 20
total_data_length = data_len * steps
total_noise_length = total_data_length - data_len + noise_len
seed4 = tf.random.normal([num_examples_to_generate_4, total_noise_length, noise_channel])
sample_data_4 = np.array([sequence_data])

In [ ]:
%%time
generated_image4 = recursive_simulator(generator, seed4, data_len, noise_len, steps)[:,:,0]
generated_image4.shape

In [ ]:
loss_obj_long = Loss(sample_data_4, real_mean=real_mean, real_std=real_std, max_lag=250, simple_loss=False, long_seq=True)
loss_obj_long.compute_loss(generated_image4)
loss_obj_long.plot_all()

In [ ]:
loss_obj_long.loss_summary

#### 800e training

In [ ]:
train(data, 2*BATCH_SIZE, generator, discriminator, generator_optimizer, discriminator_optimizer, test, 300, loss_choice)

#### test

In [ ]:
num_examples_to_generate_4 = 512
steps = 20
total_data_length = data_len * steps
total_noise_length = total_data_length - data_len + noise_len
seed4 = tf.random.normal([num_examples_to_generate_4, total_noise_length, noise_channel])
sample_data_4 = np.array([sequence_data])

In [ ]:
%%time
generated_image4 = recursive_simulator(generator, seed4, data_len, noise_len, steps)[:,:,0]
generated_image4.shape

In [ ]:
loss_obj_long = Loss(sample_data_4, real_mean=real_mean, real_std=real_std, max_lag=250, simple_loss=False, long_seq=True)
loss_obj_long.compute_loss(generated_image4)
loss_obj_long.plot_all()

In [ ]:
loss_obj_long.loss_summary

#### 1100e training

In [ ]:
train(data, 4*BATCH_SIZE, generator, discriminator, generator_optimizer, discriminator_optimizer, test, 300, loss_choice)

#### test

In [ ]:
num_examples_to_generate_4 = 512
steps = 20
total_data_length = data_len * steps
total_noise_length = total_data_length - data_len + noise_len
seed4 = tf.random.normal([num_examples_to_generate_4, total_noise_length, noise_channel])
sample_data_4 = np.array([sequence_data])

In [ ]:
%%time
generated_image4 = recursive_simulator(generator, seed4, data_len, noise_len, steps)[:,:,0]
generated_image4.shape

In [ ]:
loss_obj_long = Loss(sample_data_4, real_mean=real_mean, real_std=real_std, max_lag=250, simple_loss=False, long_seq=True)
loss_obj_long.compute_loss(generated_image4)
loss_obj_long.plot_all()

In [ ]:
loss_obj_long.loss_summary

#### 1250e training

In [ ]:
train(data, 4*BATCH_SIZE, generator, discriminator, generator_optimizer, discriminator_optimizer, test, 150, loss_choice)

#### test

In [ ]:
num_examples_to_generate_4 = 512
steps = 20
total_data_length = data_len * steps
total_noise_length = total_data_length - data_len + noise_len
seed4 = tf.random.normal([num_examples_to_generate_4, total_noise_length, noise_channel])
sample_data_4 = np.array([sequence_data])

In [ ]:
%%time
generated_image4 = recursive_simulator(generator, seed4, data_len, noise_len, steps)[:,:,0]
generated_image4.shape

In [ ]:
loss_obj_long = Loss(sample_data_4, real_mean=real_mean, real_std=real_std, max_lag=250, simple_loss=False, long_seq=True)
loss_obj_long.compute_loss(generated_image4)
loss_obj_long.plot_all()

In [ ]:
loss_obj_long.loss_summary